In [ ]:
#DeepFace를 이용하여 데이터에 bounding box를 찾음

In [ ]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tqdm.notebook import tqdm
import time

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import multiprocessing as mp
from IPython.display import clear_output

from enum import Enum

In [ ]:
!pip install pandas opencv-python tqdm 

In [ ]:
#!apt-get install -y libgl1-mesa-glx

In [ ]:
#!pip install tensorflow-gpu
#!pip install tensorflow

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
#!pip install deepface

In [ ]:
from deepface import DeepFace

In [ ]:
data_dir = os.path.join(os.getcwd(), 'train', 'images')

In [ ]:
class MaskLabels(int, Enum):
    MASK = 0
    INCORRECT = 1
    NORMAL = 2

_file_names = {
        "mask1": MaskLabels.MASK,
        "mask2": MaskLabels.MASK,
        "mask3": MaskLabels.MASK,
        "mask4": MaskLabels.MASK,
        "mask5": MaskLabels.MASK,
        "incorrect_mask": MaskLabels.INCORRECT,
        "normal": MaskLabels.NORMAL
    }

In [ ]:
# train 에 대해 bounding box 찾음

In [ ]:
check_list = []
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe'
]

processed = 0
visual = True
manualcheck = False

profiles = os.listdir(data_dir)
for profile in profiles:
    if profile.startswith("."):  # "." 로 시작하는 파일은 무시합니다
        continue

    print(processed)

    img_folder = os.path.join(data_dir, profile)
    print(img_folder)
    
    imgs = []
    bbox = []
    ages = []
    for file_name in os.listdir(img_folder):
        _file_name, ext = os.path.splitext(file_name)
        if _file_name not in _file_names:
            continue
        elif file_name[-4:] != ".jpg": # jpg 파일만 처리
            continue
        
        img_path = os.path.join(data_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
        imgs.append(np.array(Image.open(img_path)))
      
        # DeepFace
        analyzedresult = DeepFace.analyze(img_path = os.path.join(img_folder, file_name), actions = ['age'], 
                                          detector_backend = backends[4], enforce_detection = False)
        
        if analyzedresult is not None:
            bbox_coord = analyzedresult["region"]
            
            if _file_name == 'normal':
                age = analyzedresult['age']
            
            bboxfile = open(img_path.rstrip(".jpg") + ".txt", 'w')
            bboxlog = "%d, %d, %d, %d" %(bbox_coord['x'], bbox_coord['y'], bbox_coord['w'], bbox_coord['h'])
            bboxfile.write(bboxlog)
            bboxfile.close()
            
            if visual:
                bbox.append(patches.Rectangle((bbox_coord['x'], bbox_coord['y']), bbox_coord['w'], bbox_coord['h'], linewidth=3, edgecolor='b', facecolor='none'))

        
        processed = processed + 1;
    
    if visual and len(bbox) >= 7:
        n_rows, n_cols = 2, 4
        fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True, figsize=(15, 10))
        for i in range(7):
            axes[i//(n_rows+2)][i%n_cols].imshow(imgs[i])
            axes[i//(n_rows+2)][i%n_cols].add_patch(bbox[i])

        clear_output()

        print("Analyzed age : ",  age)
        plt.tight_layout()
        plt.show()
        
        if manualcheck:
            check = True
            while check:
                check_ = input('1 -> pass, 0 -> check : ')
                if check_ == '1':
                    check = False
                elif check_ == '0':
                    check_list.append(j)
                    check = False
                else:
                    pass
        


In [ ]:
_file_name

In [ ]:
data_dir = os.path.join(os.getcwd(), 'eval', 'images')

In [ ]:
# train 에 대해 bounding box 찾음

In [ ]:
processed = 0
manualcheck = False
for file_name in os.listdir(data_dir):
    
    print(processed)
    
    if file_name[0] == '.':
        continue
    elif file_name[-4:] != ".jpg": # jpg 파일만 처리
        continue

    imgs = []
    bbox = []

    img_path = os.path.join(data_dir, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
    imgs.append(np.array(Image.open(img_path)))

    # DeepFace
    analyzedresult = DeepFace.analyze(img_path = os.path.join(data_dir, file_name), actions = ['gender'], 
                                      detector_backend = backends[4], enforce_detection = False)

    if analyzedresult is not None:
        bbox_coord = analyzedresult["region"]

        bboxfile = open(img_path.rstrip(".jpg") + ".txt", 'w')
        bboxlog = "%d, %d, %d, %d" %(bbox_coord['x'], bbox_coord['y'], bbox_coord['w'], bbox_coord['h'])
        bboxfile.write(bboxlog)
        bboxfile.close()

        if visual:
            bbox.append(patches.Rectangle((bbox_coord['x'], bbox_coord['y']), bbox_coord['w'], bbox_coord['h'], linewidth=3, edgecolor='b', facecolor='none'))


    processed = processed + 1;

    if visual and len(bbox) > 0:
        fig, axes = plt.subplots()#sharex=True, sharey=True, figsize=(15, 10))

        axes.imshow(imgs[0])
        axes.add_patch(bbox[0])

        clear_output()
        plt.tight_layout()
        plt.show()

        if manualcheck:
            check = True
            while check:
                check_ = input('1 -> pass, 0 -> check : ')
                if check_ == '1':
                    check = False
                elif check_ == '0':
                    check_list.append(j)
                    check = False
                else:
                    pass

저장된 bounding box정보를 읽어서 확인.
train폴더는 7장

In [ ]:
processed = 0
visual = True
manualcheck = False

profiles = os.listdir(data_dir)
for profile in profiles:
    if profile.startswith("."):  # "." 로 시작하는 파일은 무시합니다
        continue

    print(processed)

    img_folder = os.path.join(data_dir, profile)
    print(img_folder)
    
    imgs = []
    bbox = []
    for file_name in os.listdir(img_folder):
        _file_name, ext = os.path.splitext(file_name)
        if _file_name not in _file_names:  # "." 로 시작하는 파일 및 invalid 한 파일들은 무시합니다
            continue
        elif file_name[-4:] != ".jpg": # jpg 파일만 처리
            continue
        
        img_path = os.path.join(data_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
        imgs.append(np.array(Image.open(img_path)))
      
        # DeepFace
        analyzedresult = DeepFace.analyze(img_path = os.path.join(img_folder, file_name), actions = ['gender'], 
                                          detector_backend = backends[4], enforce_detection = False)
        
        if analyzedresult is not None:
            bbox_coord = analyzedresult["region"]
            
            bboxfile = open(img_path.rstrip(".jpg") + ".txt", 'r')
            bboxlog = "%d, %d, %d, %d" %(bbox_coord['x'], bbox_coord['y'], bbox_coord['w'], bbox_coord['h'])
            bboxfile.write(bboxlog)
            bboxfile.close()
        
            if visual:
                bbox.append(patches.Rectangle((bbox_coord['x'], bbox_coord['y']), bbox_coord['w'], bbox_coord['h'], linewidth=3, edgecolor='b', facecolor='none'))

        
        processed = processed + 1;
    
    if visual and len(bbox) >= 7:
        n_rows, n_cols = 2, 4
        fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True, figsize=(15, 10))
        for i in range(7):
            axes[i//(n_rows+2)][i%n_cols].imshow(imgs[i])
            axes[i//(n_rows+2)][i%n_cols].add_patch(bbox[i])

        clear_output()
        plt.tight_layout()
        plt.show()
        
        if manualcheck:
            check = True
            while check:
                check_ = input('1 -> pass, 0 -> check : ')
                if check_ == '1':
                    check = False
                elif check_ == '0':
                    check_list.append(j)
                    check = False
                else:
                    pass